<a href="https://colab.research.google.com/github/pauljainta/Machine-Learning-Project-Adaboost-/blob/main/1605022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy scikit-learn pandas matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive/MyDrive/dataset/

[Errno 2] No such file or directory: 'gdrive/MyDrive/dataset/'
/content/gdrive/MyDrive/dataset


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
telco_customer_dataset=pd.read_csv('telco.csv')
telco_customer_dataset.drop('customerID', inplace=True, axis=1)





In [ ]:
def binarize_values(dataset):
  for i in dataset.columns:
  # print(telco_customer_dataset[i].dtypes)
    if(dataset[i].nunique()==2 and dataset[i].dtypes=='object'):
      unique_values=dataset[i].unique()
      dataset[i]=dataset[i].map({unique_values[0]:0,unique_values[1]:1})

  return dataset  


In [ ]:
def normalize(dataset):
  for i in dataset.columns:
    if(dataset[i].nunique()!=2):
      dataset[i] = (dataset[i]-dataset[i].min())/(dataset[i].max()-dataset[i].min())
  return dataset  


In [ ]:
telco_customer_dataset=binarize_values(telco_customer_dataset)
# telco_customer_dataset.head(5)

In [ ]:
telco_customer_dataset['TotalCharges'] =  pd.to_numeric(telco_customer_dataset['TotalCharges'] ,errors = 'coerce')
min_occcured_value=telco_customer_dataset['TotalCharges'].value_counts().idxmin()
  # print(adult_dataset[i].unique())
telco_customer_dataset['TotalCharges']=telco_customer_dataset['TotalCharges'].replace(np.nan,min_occcured_value)

telco_customer_dataset=pd.get_dummies(telco_customer_dataset)
telco_customer_dataset=normalize(telco_customer_dataset)

# telco_customer_dataset=telco_customer_dataset[2000:4000]



In [ ]:
X_telco=telco_customer_dataset.loc[:, telco_customer_dataset.columns != 'Churn']
Y_telco=telco_customer_dataset['Churn']
# X_telco=X_telco[1:1000]
# Y_telco=Y_telco[1:1000]


In [ ]:
adult_dataset=pd.read_csv('adult.csv')
# adult_dataset.info()
# adult_dataset.isnull().sum()


for i in adult_dataset.columns:
  # print(adult_dataset[i].dtypes)
  # adult_dataset[i].replace('?','')
  max_occcured_value=adult_dataset[i].value_counts().idxmax()
  # print(adult_dataset[i].unique())
  adult_dataset[i]=adult_dataset[i].replace(" ?",max_occcured_value)
# adult_dataset.head(400)

adult_dataset=adult_dataset[10000:20000]

In [ ]:


adult_dataset=binarize_values(adult_dataset)

adult_dataset=pd.get_dummies(adult_dataset)

# adult_dataset.head(4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
adult_dataset=normalize(adult_dataset)
adult_dataset.head(5)

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hourse-per-week,Income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Adm-clerical,...,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
10000,0.232877,0.073468,0.600000,0,0.0,0.0,0.500000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10001,0.082192,0.173768,0.800000,0,0.0,0.0,0.448980,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10002,0.506849,0.166428,0.600000,1,0.0,0.0,0.397959,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10003,0.438356,0.136557,0.866667,1,0.0,0.0,0.704082,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10004,0.109589,0.197733,0.533333,0,0.0,0.0,0.397959,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
X_adult=adult_dataset.loc[:, adult_dataset.columns != 'Income']
# X_adult
Y_adult=adult_dataset['Income']
Y_adult

10000    0
10001    0
10002    1
10003    1
10004    0
        ..
19995    1
19996    0
19997    0
19998    1
19999    1
Name: Income, Length: 10000, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
x_telco_train,x_telco_test,y_telco_train,y_telco_test = train_test_split(X_telco,Y_telco, test_size = 0.2)

x_adult_train,x_adult_test,y_adult_train,y_adult_test = train_test_split(X_adult,Y_adult, test_size = 0.2)

x_telco_test


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
5361,0,0,1,1,0.055556,1,0,0.021393,0.007610,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1
178,0,0,0,1,0.208333,1,0,0.866667,0.177758,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0
5754,0,0,0,1,0.305556,1,1,0.007960,0.050225,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0
1060,1,0,0,1,0.361111,1,0,0.017910,0.056208,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0
499,0,0,1,0,0.166667,1,0,0.810448,0.140740,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,1,0,1,0,0.263889,1,0,0.468657,0.139978,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0
998,1,0,0,0,1.000000,1,1,0.024378,0.170009,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0
4215,1,0,0,0,1.000000,1,0,0.895025,0.902585,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
1286,0,0,1,0,0.013889,1,1,0.555721,0.006381,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0


In [2]:
def hypothesis_function(x , weight):
    z = 0
    for i in range(len(weight)):
        z += np.dot(x[i],weight[i].transpose())
        # print("hypo te:"+str(np.tanh(z)))

    thz=np.tanh(z)
    if(thz > 0):
      return 1       
    return 0

In [3]:
def cost_function_derivative(weight,j,learning_rate,x_train,y_train):
    totalError = 0
    no_of_examples=len(x_train)

    for i in range(no_of_examples):
    
        y_hat = hypothesis_function(x_train.iloc[i] , weight)

        y_minus_y_hat = np.subtract((float)(y_hat),float(y_train.iloc[i]))

        error = np.dot(y_minus_y_hat,x_train.iloc[i][j])

        error += np.dot(error,1-y_hat*y_hat)
        totalError += error


    constant = float(learning_rate)/float(no_of_examples)
    return constant * totalError

In [4]:
def gradient_descent(weight,learning_rate,x_train,y_train):
    new_weight = []
    no_of_examples=len(x_train)

    for j in range(len(weight)):
        new_weight.append(weight[j] - cost_function_derivative(weight,j,learning_rate,x_train,y_train))
        
    return new_weight

In [5]:
def logistic_regression(learning_rate,weight,num_iters,x_train,x_test,y_train,y_test):
    no_of_examples = len(y_train)

    for x in range(num_iters):
        print("Logistic Regression iteration:"+str(x))
        weight = gradient_descent(weight,learning_rate,x_train,y_train)
        
       
    score = 0
    

    for i in range(len(y_test)):
        prediction = round(hypothesis_function(x_test.iloc[i] , weight))
        # print("Predicion:"+str(prediction))
        answer = y_test.iloc[i]
        score += ( 1 - abs(prediction-answer))
        # print("Score:"+str(score))
        
    accuracy = float(score) / float(length)
    print ('Logistic Regression Accuracy: ', accuracy)

    return weight

In [6]:
def Adaboosting(x_train,x_test,y_train,y_test,learning_rate,num_of_rounds,num_of_iterations,weight_init):

  no_of_examples = len(x_train)

  weight_of_samples = np.empty(no_of_examples)
  weight_of_samples.fill(1.0/float(no_of_examples))
  
  logistic_regression_weight_container = []
  hypothesis_weight = []

  np.random.seed(5)

 

  for i in range(num_of_rounds):
    
    resampled_set_of_examples = np.random.choice(x_train.shape[0], size=no_of_examples, p=weight_of_samples)
    
    x_train = x_train.iloc[resampled_set_of_examples]
    y_train = y_train.iloc[resampled_set_of_examples]

    logistic_regression_weight = logistic_regression(learning_rate,weight_init,num_of_iterations,x_train,x_test,y_train,y_test)
    logistic_regression_weight_container.append(logistic_regression_weight)

    error = 0

    for j in range(no_of_examples):
      y_hat = hypothesis_function(x_train.iloc[j],logistic_regression_weight)
      if y_hat != y_train.iloc[j]:
        error = error + weight_of_samples[j]


    if(error < 0.5):
      for j in range(no_of_examples):
        y_hat = hypothesis_function(x_train.iloc[j],logistic_regression_weight)
        if y_hat == y_train.iloc[j]:
          weight_of_samples[j] = weight_of_samples[j] * float(error)/(1.0-float(error))

      weight_of_samples = (weight_of_samples-weight_of_samples.min())/(weight_of_samples.max()-weight_of_samples.min())
      hypothesis_weight.append(np.log2(  (1.0-float(error))   /   float(error)   ))


    else:
      continue



In [ ]:
weight_init = np.zeros(x_telco_test.iloc[1].shape)
learning_rate = 0.5
iterations = 10
# x_telco_test.shape
logistic_regression(learning_rate,weight_init,iterations,x_telco_train,x_telco_test,y_telco_train,y_telco_test)

iteration:0
iteration:1
iteration:2
iteration:3
iteration:4
iteration:5
iteration:6
iteration:7
iteration:8
iteration:9
Predicion:0
Score:1
Predicion:0
Score:2
Predicion:0
Score:3
Predicion:0
Score:4
Predicion:0
Score:4
Predicion:0
Score:5
Predicion:0
Score:6
Predicion:0
Score:7
Predicion:0
Score:8
Predicion:0
Score:9
Predicion:0
Score:10
Predicion:0
Score:11
Predicion:0
Score:12
Predicion:0
Score:13
Predicion:1
Score:14
Predicion:0
Score:15
Predicion:0
Score:16
Predicion:0
Score:17
Predicion:0
Score:18
Predicion:0
Score:19
Predicion:0
Score:20
Predicion:0
Score:21
Predicion:0
Score:22
Predicion:1
Score:23
Predicion:1
Score:24
Predicion:0
Score:25
Predicion:1
Score:25
Predicion:0
Score:26
Predicion:0
Score:27
Predicion:0
Score:27
Predicion:0
Score:28
Predicion:0
Score:29
Predicion:0
Score:30
Predicion:1
Score:31
Predicion:0
Score:32
Predicion:0
Score:33
Predicion:0
Score:34
Predicion:0
Score:35
Predicion:0
Score:36
Predicion:0
Score:36
Predicion:0
Score:37
Predicion:0
Score:38
Predicio